https://download.bls.gov/pub/time.series/ap/

In [1]:
%use dataframe
%use kandy

In [2]:
import java.nio.file.Files
import java.nio.file.Path

fun String.writeToFile(path: Path): Path {
    path.parent.toFile().mkdirs()
    Files.writeString(path, this)
    return path;
}

In [3]:
import java.nio.file.Paths
import kotlin.io.path.inputStream

val itemCode by column<String>("item_code")
val itemName by column<String>("item_name")

val itemIndex =
    DataFrame.readTSV(Paths.get("raw-data").resolve("consumer-prices").resolve("item-dictionary.txt").inputStream())
        .map {
            Pair(itemCode.getValue(it), itemName.getValue(it))
        }
        .toMap()

In [4]:
val seriesId by column<String>("series_id")
val areaCode by column<String>("area_code")
val seriesTitle by column<String>("series_title")

data class SeriesDetails(val areaCode: String, val itemCode: String, val seriesTitle: String)

val seriesIndex =
    DataFrame.readTSV(Paths.get("raw-data").resolve("consumer-prices").resolve("series-dictionary.txt").inputStream())
        .map {
            Pair(
                seriesId.getValue(it),
                SeriesDetails(areaCode.getValue(it), itemCode.getValue(it), seriesTitle.getValue(it))
            )
        }
        .toMap()

In [5]:
val year by column<Int>("year")
val period by column<String>("period")
val value by column<Double>("value")
val valueAsString by column<String>("value")
val product by column<String>()
val month by column<String>()
val date by column<String>()

DataFrame.readTSV(Paths.get("raw-data").resolve("consumer-prices").resolve("data.txt").inputStream())
    .filter { year.getValue(it) >= 2016 }
    .filter { year.getValue(it) <= 2024 }
    .filter { valueAsString.getValue(it).toDoubleOrNull() != null }
    .add(product) {
        seriesIndex[seriesId.getValue(it)]?.seriesTitle
    }
    .add(areaCode) {
        seriesIndex[seriesId.getValue(it)]?.areaCode
    }
    .add(month) {
        period.getValue(it).replace("M", "")
    }
    .filter { year.getValue(it) > 2016 || (year.getValue(it) == 2016 && month.getValue(it).equals("12")) }
    .add(date) {
        "${month.getValue(it)}/${year.getValue(it)}"
    }
    .convert(value).to<Double>()
    .toCsv()
    .writeToFile(Paths.get("presentable-data").resolve("consumer-prices").resolve("consumer-prices-2017-2024.csv"))

presentable-data/consumer-prices/consumer-prices-2017-2024.csv

In [6]:
val count by column<Int>()

val frame = DataFrame.readCSV(
    Paths.get("presentable-data").resolve("consumer-prices").resolve("consumer-prices-2017-2024.csv").inputStream()
)
    .filter {
        areaCode.getValue(it).equals("0000")
    }
    .groupBy(seriesId, product)
    .aggregate {
        count() into count
    }.sortByDesc(count)
    .filter { count.getValue(it) >= 88 }
val validSeries =  frame .select(seriesId)
    .map{
        seriesId.getValue(it)
    }
    .toList()
    .toSet()
    
frame.count()

49

In [7]:
var textHeight = 8

DataFrame.readCSV(
    Paths.get("presentable-data").resolve("consumer-prices").resolve("consumer-prices-2017-2024.csv").inputStream()
)
    .filter {
        setOf(
            "APU0000704111",
            "APU0000708111",
            "APU0000709112",
            "APU00007471A",
            "APU0000FF1101"
        ).contains(seriesId.getValue(it))
    }
    .plot {
        y.axis.name = "Price(in $)"
        x.axis.name = "Month/Year"
        x(date)
        line {
            y(value)
            color(product) {
                legend.type = LegendType.DiscreteLegend(nCol = 1, byRow = true)
                legend.name = "Type of product"
            }
        }
        vLine {
            xIntercept.constant(1)
            color = Color.RED
            type = LineType.DOTTED
        }
        text {
            x.constant(3)
            y.constant(textHeight)
            label = "Trump elected"
        }
        vLine {
            xIntercept.constant(39)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(43)
            y.constant(textHeight)
            label = "Covid-19 becomes pandemic"
        }
        vLine {
            xIntercept.constant(49)
            color = Color.BLUE
            type = LineType.DOTTED
        }
        text {
            x.constant(51)
            y.constant(textHeight)
            label = "Biden elected"
        }
        vLine {
            xIntercept.constant(61)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(64)
            y.constant(textHeight)
            label = "Omicron variant"
        }
        vLine {
            xIntercept.constant(77)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(81)
            y.constant(textHeight)
            label = "End of Covid-19 pandemic"
        }
        layout.size = Pair(3000, 700)
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="IvmvBq"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"date":["12/2016","01/2017","02/2017","03/2017","04/2017","05/2017","06/2017","07/2017","08/2017","09/2017","10/2017","11/2017","12/2017","01/2018","02/2018","03/2018","04/2018","05/2018","06/2018","07/2018","08/2018","09/2018","10/2018","11/2018","12/2018","01/2019","02/2019","03/2019","04/2019","05/2019","06/2019","07/2019","08/2019","09/2019","10/2019","11/2019","12/2019","01/2020","02/2020","03/2020","04/2020","05/2020","06/2020","07/2020","08/2020","09/2020","10/2020","11/2020","12/2020","01/2021","02/2021","03/2021","04/2021","05/2021","06/2021","07/2021","08/2021","09/2021","10/2021","11/2021","12/2021","01/2022","02/2022","03/2022","04/2022","05/2022","06/2022","07/2022","08/2022","09/2022","10/2022","11/2022","12/2022","01/2023","02/2023","03/2023","04/2023","05/2023","06/2023","07/2023","08/2023","09/2023","10/2023","11/2023","12/2023","01/2024","02/2024","03/2024","04/2024","05/2024","12/2016","01/2017","02/2017","03/2017","04/2017","05/2017","06/2017","07/2017","08/2017","09/2017","10/2017","11/2017","12/2017","01/2018","02/2018","03/2018","04/2018","05/2018","06/2018","07/2018","08/2018","09/2018","10/2018","11/2018","12/2018","01/2019","02/2019","03/2019","04/2019","05/2019","06/2019","07/2019","08/2019","09/2019","10/2019","11/2019","12/2019","01/2020","02/2020","03/2020","04/2020","05/2020","06/2020","07/2020","08/2020","09/2020","10/2020","11/2020","12/2020","01/2021","02/2021","03/2021","04/2021","05/2021","06/2021","07/2021","08/2021","09/2021","10/2021","11/2021","12/2021","01/2022","02/2022","03/2022","04/2022","05/2022","06/2022","07/2022","08/2022","09/2022","10/2022","11/2022","12/2022","01/2023","02/2023","03/2023","04/2023","05/2023","06/2023","07/2023","08/2023","09/2023","10/2023","11/2023","12/2023","01/2024","02/2024","03/2024","04/2024","05/2024","12/2016","01/2017","02/2017","03/2017","04/2017","05/2017","06/2017","07/2017","08/2017","09/2017","10/2017","11/2017","12/2017","01/2018","02/2018","03/2018","04/2018","05/2018","06/2018","07/2018","08/2018","09/2018","10/2018","11/2018","12/2018","01/2019","02/2019","03/2019","04/2019","05/2019","06/2019","07/2019","08/2019","09/2019","10/2019","11/2019","12/2019","01/2020","02/2020","03/2020","04/2020","05/2020","06/2020","07/2020","08/2020","09/2020","10/2020","11/2020","12/2020","01/2021","02/2021","03/2021","04/2021","05/2021","06/2021","07/2021","08/2021","09/2021","10/2021","11/2021","12/2021","01/2022","02/2022","03/2022","04/2022","05/2022","06/2022","07/2022","08/2022","09/2022","10/2022","11/2022","12/2022","01/2023","02/2023","03/2023","04/2023","05/2023","06/2023","07/2023","08/2023","09/2023","10/2023","11/2023","12/2023","01/2024","02/2024","03/2024","04/2024","05/2024","12/2016","01/2017","02/2017","03/2017","04/2017","05/2017","06/2017","07/2017","08/2017","09/2017","10/2017","11/2017","12/2017","01/2018","02/2018","03/2018","04/2018","05/2018","06/2018","07/2018","08/2018","09/2018","10/2018","11/2018","12/2018","01/2019","02/2019","03/2019","04/2019","05/2019","06/2019","07/2019","08/2019","09/2019","10/2019","11/2019","12/2019","01/2020","02/2020","03/2020","04/2020","05/2020","06/2020","07/2020","08/2020","09/2020","10/2020","11/2020","12/2020","01/2021","02/2021","03/2021","04/2021","05/2021","06/2021","07/2021","08/2021","09/2021","10/2021","11/2021","12/2021","01/2022","02/2022","03/2022","04/2022","05/2022","06/2022","07/2022","08/2022","09/2022","10/2022","11/2022","12/2022","01/2023","02/2023","03/2023","04/2023","05/2023","06/2023","07/2023","08/2023","09/2023","10/2023","11/2023","12/2023","01/2024","02/2024","03/2024","04/20

Z

In [8]:
val inflation by column<Double>()

textHeight = 4
DataFrame.readCSV(Paths.get("presentable-data").resolve("consumer-prices").resolve("consumer-prices-2017-2024.csv").inputStream())
    .filter {
        validSeries.contains(seriesId.getValue(it))
    }
    .add(inflation){
        if(date.getValue(it).equals("12/2016")){
            0.0
        }else{
            var previousMonthValue = it.prev()?.get(value)!!
            val currentMonthValue = value.getValue(it)
            100.0 - (previousMonthValue / (currentMonthValue / 100))
        }
    }
    .groupBy(date,year,month)
    .aggregate { 
        mean(value) into value
        mean(inflation) into inflation
    }
    .sortBy(year,month)
    .plot {
        y.axis.name = "Month to Month change in %"
        y.axis{
            breaks(format = "{} %")
        }
        x.axis.name = "Month/Year"
        x(date)
        bars{
            y(inflation)
        }
        vLine {
            xIntercept.constant(1)
            color = Color.RED
            type = LineType.DOTTED
        }
        text {
            x.constant(3)
            y.constant(textHeight)
            label = "Trump elected"
        }
        vLine {
            xIntercept.constant(39)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(43)
            y.constant(textHeight)
            label = "Covid-19 becomes pandemic"
        }
        vLine {
            xIntercept.constant(49)
            color = Color.BLUE
            type = LineType.DOTTED
        }
        text {
            x.constant(51)
            y.constant(textHeight)
            label = "Biden elected"
        }
        vLine {
            xIntercept.constant(61)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(64)
            y.constant(textHeight)
            label = "Omicron variant"
        }
        vLine {
            xIntercept.constant(77)
            color = Color.YELLOW
            type = LineType.DOTTED
        }
        text {
            x.constant(81)
            y.constant(textHeight)
            label = "End of Covid-19 pandemic"
        }
        layout.size = Pair(3000, 700)
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bNs5Qp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"date":["12/2016","01/2017","02/2017","03/2017","04/2017","05/2017","06/2017","07/2017","08/2017","09/2017","10/2017","11/2017","12/2017","01/2018","02/2018","03/2018","04/2018","05/2018","06/2018","07/2018","08/2018","09/2018","10/2018","11/2018","12/2018","01/2019","02/2019","03/2019","04/2019","05/2019","06/2019","07/2019","08/2019","09/2019","10/2019","11/2019","12/2019","01/2020","02/2020","03/2020","04/2020","05/2020","06/2020","07/2020","08/2020","09/2020","10/2020","11/2020","12/2020","01/2021","02/2021","03/2021","04/2021","05/2021","06/2021","07/2021","08/2021","09/2021","10/2021","11/2021","12/2021","01/2022","02/2022","03/2022","04/2022","05/2022","06/2022","07/2022","08/2022","09/2022","10/2022","11/2022","12/2022","01/2023","02/2023","03/2023","04/2023","05/2023","06/2023","07/2023","08/2023","09/2023","10/2023","11/2023","12/2023","01/2024","02/2024","03/2024","04/2024","05/2024"],
"inflation":[0.0,0.6042381086240344,-0.1485332759657711,-0.0013958434965643948,0.18603423066984676,0.0372953000268964,0.46668429260391636,0.29764308917010124,0.4161038277772328,0.8270791693138454,-1.058652000860095,0.3175970239905465,0.6645741015846636,-0.963008376869306,-0.3290962948516447,-0.09920913563299925,1.3499935741473974,-0.24323944879442128,-0.22145824601126807,0.3588186911844827,-0.10480668553618838,-0.044519610549064725,-0.20368685742218795,-0.5428624529494748,-0.5575354473584109,-0.3573026702333104,0.24712317374472867,0.5802663247275001,0.8042865965791753,0.04709026351316443,-0.8326943445592069,0.38932412752005635,-0.770034803943613,0.2798986284655285,-0.14040919193875562,0.5447738527232078,0.7457482625475872,-0.010060770639994974,-0.4223088184615286,-0.8856053920799071,1.1741527007216643,2.6427479351308634,2.51725209115731,-2.4564468308934773,-1.5035446104479568,-0.7940159880650047,0.24929907690939762,0.623294823213827,0.805858851783609,0.8217048218169588,0.7701809019905647,1.408915721964383,0.9357225513600913,1.066522328033046,1.130964514199171,0.42217174635565113,0.6177481347878429,1.3944937676205331,1.9917770730974422,0.6956261057258029,-0.7444033938514121,0.9785922230209113,1.252792497280946,3.250636024237205,1.5530827415742643,1.8333330379328356,1.710259140554777,-0.02333351614247308,-0.14697542938502875,-0.09895904925569361,1.087790210152492,-0.6441498202399858,-1.1062242188907643,0.29243870936902255,-0.5224408473802967,-1.2108255561680668,0.5520297237995534,-0.47418711015295634,-0.1613911439935964,0.8305417100535962,1.4339314056524501,0.32586710864219404,0.15874300783746448,-1.2488327916535298,-0.18054182107383535,0.1761331698493147,0.22934276736763495,0.45649838821289335,0.3179445062311155,-0.17475980786375467]
},
"ggsize":{
"width":3000.0,
"height":700.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x

Cannot be inflation adjusted because the same data is used in calculating CPI which is what most of the inflation % are calculated from(change of price on basket of goods)

In [9]:
val dataFrame = DataFrame.readCSV(
    Paths.get("presentable-data").resolve("consumer-prices").resolve("consumer-prices-2017-2024.csv").inputStream()
)
    .filter {
        validSeries.contains(seriesId.getValue(it))
    }
    .add(inflation) {
        if (date.getValue(it).equals("12/2016")) {
            0.0
        } else {
            var previousMonthValue = it.prev()?.get(value)!!
            val currentMonthValue = value.getValue(it)
            100.0 - (previousMonthValue / (currentMonthValue / 100))
        }
    }
    .groupBy(date, year, month)
    .aggregate {
        mean(value) into value
        mean(inflation) into inflation
    }
    .sortBy(year, month)

val trumpEra = dataFrame
    .filter {
        year.getValue(it) > 2016 && year.getValue(it) <= 2020
    }

val trumpPricesGrew = trumpEra.filter { inflation.getValue(it) > 0.0 }.count()
val trumpPricesShrunk = trumpEra.filter { inflation.getValue(it) <= 0.0 }.count()

println("Trump : Up $trumpPricesGrew, Down $trumpPricesShrunk, Total ${trumpPricesGrew + trumpPricesShrunk}")

val bidenEra = dataFrame
    .filter {
        year.getValue(it) > 2020
    }

val bidenPricesGrew = bidenEra.filter { inflation.getValue(it) > 0.0 }.count()
val bidenPricesShrunk = bidenEra.filter { inflation.getValue(it) <= 0.0 }.count()

println("Biden : Up $bidenPricesGrew, Down $bidenPricesShrunk, Total ${bidenPricesGrew + bidenPricesShrunk}")

val monthAsInt by column<Int>("month")

val trumpEraWithoutCovid = dataFrame
    .convert(month).to<Int>()
    .filter {
        year.getValue(it) < 2020 || (year.getValue(it) == 2020 && monthAsInt.getValue(it) < 3)
    }

val trumpPricesGrewWithoutCovid = trumpEraWithoutCovid.filter { inflation.getValue(it) > 0.0 }.count()
val trumpPricesShrunkWithoutCovid = trumpEraWithoutCovid.filter { inflation.getValue(it) <= 0.0 }.count()

println("Trump : Up $trumpPricesGrewWithoutCovid, Down $trumpPricesShrunkWithoutCovid, Total ${trumpPricesGrewWithoutCovid + trumpPricesShrunkWithoutCovid}")

val bidenEraWithoutCovid = dataFrame
    .convert(month).to<Int>()
    .filter {
        year.getValue(it) > 2023 || (year.getValue(it) == 2023 && monthAsInt.getValue(it) > 5)
    }

val bidenPricesGrewWithoutCovid = bidenEraWithoutCovid.filter { inflation.getValue(it) > 0.0 }.count()
val bidenPricesShrunkWithoutCovid = bidenEraWithoutCovid.filter { inflation.getValue(it) <= 0.0 }.count()

println("Biden : Up $bidenPricesGrewWithoutCovid, Down $bidenPricesShrunkWithoutCovid, Total ${bidenPricesGrewWithoutCovid + bidenPricesShrunkWithoutCovid}")

 

Trump : Up 25, Down 23, Total 48
Biden : Up 28, Down 13, Total 41
Trump : Up 19, Down 20, Total 39
Biden : Up 8, Down 4, Total 12
